In [ ]:
from pathlib import Path
from asim.dataset.scene.arrow_scene import ArrowScene
from typing import Optional, Tuple, Union

import matplotlib.pyplot as plt

import numpy as np

from asim.common.geometry.base import Point2D
from asim.common.geometry.bounding_box.bounding_box import BoundingBoxSE2, BoundingBoxSE3
from asim.common.geometry.tranform_2d import translate_along_yaw
from asim.common.visualization.color.color import ELLIS_5
from asim.dataset.scene.abstract_scene import AbstractScene

from asim.common.visualization.matplotlib.map import add_default_map_on_ax
from asim.common.visualization.matplotlib.observation import add_box_detections_to_ax, add_ego_vehicle_to_ax, add_traffic_lights_to_ax

from asim.common.geometry.base import Point3D

from asim.common.visualization.color.default import BOX_DETECTION_CONFIG


In [ ]:



def plot_scene_at_iteration(scene: AbstractScene, iteration: int = 0, ax: Optional[plt.Axes] = None):

    radius = 100

    ego_vehicle_state = scene.get_ego_vehicle_state_at_iteration(iteration)
    box_detections = scene.get_box_detections_at_iteration(iteration)
    traffic_light_detections = scene.get_traffic_light_detections_at_iteration(iteration)
    map_api = scene.map_api

    point_2d = ego_vehicle_state.bounding_box.center.state_se2.point_2d

    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 10))
    add_default_map_on_ax(ax, map_api, point_2d, radius=radius)
    add_traffic_lights_to_ax(ax, traffic_light_detections, map_api)

    add_box_detections_to_ax(ax, box_detections)
    add_ego_vehicle_to_ax(ax, ego_vehicle_state)

    buffer_radius = 0.8 * radius
    ax.set_xlim(point_2d.x - buffer_radius, point_2d.x + buffer_radius)
    ax.set_ylim(point_2d.y - buffer_radius, point_2d.y + buffer_radius)

    ax.set_aspect("equal", adjustable="box")

    return ax


log_name = "2021.06.07.12.54.00_veh-35_01843_02314"
log_file = Path(f"/home/daniel/asim_workspace/data/mini/{log_name}.arrow")
scene = ArrowScene(log_file)

plot_scene_at_iteration(scene, iteration=3000)

In [ ]:
from asim.common.geometry.base import StateSE3
from asim.common.visualization.color.color import Color
from asim.common.visualization.color.config import PlotConfig
import trimesh
from scipy.spatial.transform import Rotation

def bounding_box_to_trimesh(bbox: BoundingBoxSE3, plot_config: PlotConfig) -> trimesh.Trimesh:
    """
    Convert a 3D bounding box to a trimesh object.

    Args:
        bbox: BoundingBoxSE3 object containing center pose and dimensions

    Returns:
        trimesh.Trimesh: A box mesh representing the bounding box
    """
    # Create a unit box centered at origin
    box_mesh = trimesh.creation.box(extents=[bbox.length, bbox.width, bbox.height])

    # Create rotation matrix from roll, pitch, yaw (intrinsic rotations)
    # Using 'xyz' convention: roll (x), pitch (y), yaw (z)
    rotation = Rotation.from_euler("xyz", [bbox.center.roll, bbox.center.pitch, bbox.center.yaw])
    rotation_matrix = rotation.as_matrix()

    # Create 4x4 transformation matrix
    transform_matrix = np.eye(4)
    transform_matrix[:3, :3] = rotation_matrix
    transform_matrix[:3, 3] = [bbox.center.x, bbox.center.y, bbox.center.z]

    # Apply transformation to the box
    box_mesh.apply_transform(transform_matrix)

    base_color = [r/255.0 for r in plot_config.fill_color.rgba]
    box_mesh.visual.face_colors = plot_config.fill_color.rgba

    # # Create material with reduced shininess
    # material = trimesh.visual.material.SimpleMaterial(
    #     diffuse=plot_config.fill_color.rgba,        # Main color of the object
    #     ambient=[30, 30, 30, 255],        # Low ambient light reflection
    #     specular=[50, 50, 50, 255],       # Low specular reflection (reduces shininess)
    #     glossiness=10.0                     # Low glossiness value (0-100, lower = less shiny)
    # )

    # Apply the material to your mesh
    # box_mesh.visual.material = material

    # Alternative: If you want even less shine, you can set specular to very low values
    # specular = 100
    # ambient = 100
    # glossiness = 0
    # alpha = 255
    # material_matte = trimesh.visual.material.SimpleMaterial(
    #     diffuse=plot_config.fill_color.rgba,
    #     ambient=[ambient, ambient, ambient, alpha],
    #     specular=[specular, specular, specular, alpha],       
    #     glossiness=glossiness                      
    # )

    # # Apply the matte material
    # box_mesh.visual.material = material_matte



    pbr_material = trimesh.visual.material.PBRMaterial(
        baseColorFactor=base_color,         # Your desired color (RGBA, 0-1 range)
        metallicFactor=1.0,                 # 0.0 = non-metallic (more matte)
        roughnessFactor=0.9,                # 0.8 = quite rough (less shiny, 0=mirror, 1=completely rough)
        emissiveFactor=[0.0, 0.0, 0.0],    # No emission
        alphaCutoff=0.75,                    # Alpha threshold for transparency
        doubleSided=False                   # Single-sided material
    )
    box_mesh.visual.material = pbr_material

    return box_mesh


def bounding_box_to_trimesh_alternative(bbox: BoundingBoxSE3) -> trimesh.Trimesh:
    """
    Alternative implementation using trimesh's built-in transformation methods.

    Args:
        bbox: BoundingBoxSE3 object containing center pose and dimensions

    Returns:
        trimesh.Trimesh: A box mesh representing the bounding box
    """
    # Create a unit box centered at origin
    box_mesh = trimesh.creation.box(extents=[bbox.length, bbox.width, bbox.height])

    # Apply rotations in order: roll, pitch, yaw
    box_mesh = box_mesh.apply_transform(trimesh.transformations.rotation_matrix(bbox.center.roll, [1, 0, 0]))
    box_mesh = box_mesh.apply_transform(trimesh.transformations.rotation_matrix(bbox.center.pitch, [0, 1, 0]))
    box_mesh = box_mesh.apply_transform(trimesh.transformations.rotation_matrix(bbox.center.yaw, [0, 0, 1]))

    # Apply translation
    box_mesh = box_mesh.apply_translation([bbox.center.x, bbox.center.y, bbox.center.z])

    return box_mesh





In [ ]:
import viser


server = viser.ViserServer()

log_name = "2021.06.07.12.54.00_veh-35_01843_02314"
log_file = Path(f"/home/daniel/asim_workspace/data/mini/{log_name}.arrow")
scene = ArrowScene(log_file)

# server.scene.add_light_hemisphere(name="ambient_light", sky_color=ELLIS_5[0].rgb, ground_color=ELLIS_5[1].rgb, intensity=1.0)
# server.scene.add_light_spot("test")

In [ ]:



from asim.common.visualization.color.default import EGO_VEHICLE_CONFIG


def translate_bounding_box_se3(bounding_box_se3: BoundingBoxSE3, point_3d: Point3D) -> BoundingBoxSE3:
    bounding_box_se3.center.x = bounding_box_se3.center.x - point_3d.x
    bounding_box_se3.center.y = bounding_box_se3.center.y - point_3d.y
    bounding_box_se3.center.z = bounding_box_se3.center.z - point_3d.z
    return bounding_box_se3


iteration = 100

ego_vehicle_state = scene.get_ego_vehicle_state_at_iteration(iteration)
box_detections = scene.get_box_detections_at_iteration(iteration)
traffic_light_detections = scene.get_traffic_light_detections_at_iteration(iteration)
map_api = scene.map_api

center = ego_vehicle_state.bounding_box.center.point_3d
for box_detection in box_detections:
    bbox: BoundingBoxSE3 = box_detection.bounding_box
    bbox = translate_bounding_box_se3(bbox, center)
    plot_config = BOX_DETECTION_CONFIG[box_detection.metadata.detection_type]
    trimesh_box = bounding_box_to_trimesh(bbox, plot_config)
    server.scene.add_mesh_trimesh(name=f"{box_detection.metadata.detection_type.serialize()}/{box_detection.metadata.track_token}", mesh=trimesh_box,)

ego_bbox = translate_bounding_box_se3(ego_vehicle_state.bounding_box, center)
trimesh_box = bounding_box_to_trimesh(ego_bbox, EGO_VEHICLE_CONFIG)
server.scene.add_mesh_trimesh(name="ego", mesh=trimesh_box)

In [ ]:
from typing import List
from asim.dataset.maps.map_datatypes import MapSurfaceType


layers: List[MapSurfaceType] = [
    MapSurfaceType.LANE,
    MapSurfaceType.LANE_GROUP,
    MapSurfaceType.GENERIC_DRIVABLE,
    MapSurfaceType.CARPARK,
    MapSurfaceType.CROSSWALK,
    MapSurfaceType.INTERSECTION,
    MapSurfaceType.WALKWAY,
]

# radius = 50
# map_objects_dict = map_api.get_proximal_map_objects(center.point_2d, radius=radius, layers=layers)
# for layer, map_objects in map_objects_dict.items():
#     for map_object in map_objects:
#         if layer in [
#             MapSurfaceType.LANE_GROUP,
#             MapSurfaceType.GENERIC_DRIVABLE,
#             MapSurfaceType.CARPARK,
#             MapSurfaceType.CROSSWALK,
#             MapSurfaceType.INTERSECTION,
#             MapSurfaceType.WALKWAY,
#         ]:
#             add_shapely_polygon_to_ax(ax, map_object.shapely_polygon, MAP_SURFACE_CONFIG[layer])
#         if layer in [MapSurfaceType.LANE]:
#             map_object: AbstractLane
#             add_shapely_linestring_to_ax(ax, map_object.centerline.linestring, CENTERLINE_CONFIG)
#             # ax.plot(*map_object.centerline.array[:, :2].T, color="grey", alpha=1.0, linestyle="dashed", zorder=2)

In [ ]:
import numpy as np
import trimesh
from shapely.geometry import LineString

from asim.common.visualization.color.default import MAP_SURFACE_CONFIG
from asim.dataset.maps.abstract_map_objects import AbstractLane

def linestring_to_trimesh(linestring, point_3d: Point3D, fill_holes=True):
    """Convert a 3D LineString to a trimesh surface."""
    # Extract coordinates from the linestring
    translate = point_3d.array.copy()
    translate[2] = 1.5  # Ignore Z coordinate for 2D projection

    coords = np.array(linestring) - translate
    
    # For a closed surface, ensure the last point connects to the first
    if not np.allclose(coords[0], coords[-1]):
        coords = np.vstack([coords, coords[0]])
    
    # Create a 2D projection for triangulation (using X,Y coordinates)
    points_2d = coords[:, :2]
    
    # Perform Delaunay triangulation on the 2D projection
    from scipy.spatial import Delaunay
    tri = Delaunay(points_2d)
    
    # Create the mesh using original 3D coordinates
    mesh = trimesh.Trimesh(vertices=coords, faces=tri.simplices)
    
    if fill_holes:
        mesh.fill_holes()
    
    mesh.visual.face_colors = MAP_SURFACE_CONFIG[MapSurfaceType.LANE].fill_color.rgba

    return mesh


radius = 50
map_objects_dict = map_api.get_proximal_map_objects(center.point_2d, radius=radius, layers=[MapSurfaceType.LANE])
for lane in map_objects_dict[MapSurfaceType.LANE]:
    lane: AbstractLane

    right_boundary = lane.right_boundary.array
    left_boundary = lane.left_boundary.array
    outline = np.vstack([right_boundary, left_boundary[::-1]])
    
    trimesh_surface = linestring_to_trimesh(outline, center)
    server.scene.add_mesh_trimesh(name=f"lane/{lane.id}", mesh=trimesh_surface)